In [2]:
import numpy as np
import os
import math
import pandas as pd
from datetime import date as pdate
from sklearn import linear_model
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import statistics

In [3]:
# demo of rank reduction of a correlation matrix

filepath = "./data/sofr_hist_curves2.csv" 

df_data = pd.read_csv(filepath, index_col = ["Tenor"]).T 
corr_matrix = df_data.corr()

egval, egvec = np.linalg.eig(corr_matrix)

egval[0], egvec[0]

(23.24993421972503,
 array([-1.85015760e-01, -5.01612288e-01, -6.54240592e-01, -2.17516006e-01,
         4.32883406e-01, -2.00905550e-01,  1.61403996e-02,  9.01732099e-02,
         3.85052105e-02,  1.59034313e-02, -2.23328962e-02,  2.65428895e-03,
        -2.03718221e-02,  2.50960065e-03,  2.12911553e-03,  5.36264180e-04,
         1.13497929e-05,  4.92228838e-05,  3.73190377e-06,  3.63584925e-05,
         3.63554845e-05,  4.11688081e-05,  1.57374189e-05, -3.01937152e-05]))

In [29]:
# target rank m
m=5
df_eigenvec = pd.DataFrame(egvec[:][:,0:m])     # E 
df_eigenval_sqrt = pd.DataFrame(np.sqrt(np.diag(egval[0:m])).tolist())  # sqrt(lambda)
factor_loadings = pd.DataFrame(np.dot(df_eigenvec, df_eigenval_sqrt).tolist()) # H hat matrix
norms = pd.DataFrame(np.dot(factor_loadings, factor_loadings.T).tolist()) # A matrix


In [ ]:

rescaling = np.diag(np.reciprocal(np.sqrt(np.diag(norms.to_numpy()))))  # C matrix for scaling
factor_loadings_rescaled = pd.DataFrame(np.dot(rescaling, np.dot(df_eigenvec, df_eigenval_sqrt).tolist()))  # H hat scaled = sqrt(C) * E hat * sqrt(lambda hat)
corr_matrix_approx = pd.DataFrame(np.dot(factor_loadings_rescaled, factor_loadings_rescaled.T).tolist())    # A hat - new rank reduced correlation
#print(corr_matrix_approx)
#print(corr_matrix)
print(pd.DataFrame(corr_matrix_approx.to_numpy() - corr_matrix.to_numpy()))

egval

In [17]:
rescaling

array([[1.00000864, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.00007191, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.00001806, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.   

In [45]:
# eigen vectors in one matrix
E_hat = np.array(
    [[0.5703, 0.2977, -.1687],
     [0.4716, 0.6229, 0.3192],
     [0.512, -0.3821, -.6639],
     [0.4361, -.6143, .6549]]
)
# eigen values in one matrix
lambda_hat = np.diag(np.array([2.8, .8889, .3078]))
lambda_hat

array([[2.8   , 0.    , 0.    ],
       [0.    , 0.8889, 0.    ],
       [0.    , 0.    , 0.3078]])

In [48]:
# compute the factor loadings or new H or Principal Components (PC)
H_hat = E_hat @ np.sqrt(lambda_hat)
H_hat @ H_hat.T     # display the resultant correlation matrix

array([[0.99821676, 0.90133025, 0.75094224, 0.49981622],
       [0.90133025, 0.99899681, 0.39929058, 0.30006976],
       [0.75094224, 0.39929058, 0.99944988, 0.70001137],
       [0.49981622, 0.30006976, 0.70001137, 0.99996586]])

In [ ]:
# Notice above does not have 1 for the variances whereas the data is standardised e.g. X_1 ~ N(0,1)
# so it should be 1. Perform the following to normalise the corr matrix
norms = H_hat @ H_hat.T
# find scaling matrix
rescaling = np.diag(np.reciprocal(np.sqrt(np.diag(norms))))
# scale H_hat and finc correlation matrix again
H_hat_scaled = rescaling @ H_hat
H_hat_scaled @ H_hat_scaled.T

array([[1.        , 0.90258782, 0.75181952, 0.500271  ],
       [0.90258782, 1.        , 0.39960094, 0.30022551],
       [0.75181952, 0.39960094, 1.        , 0.70021595],
       [0.500271  , 0.30022551, 0.70021595, 1.        ]])